### Initial Chromosome Structure

Initially I will deal with a simplified verison of the problem. I'm simply going to look to produce a feasible solution, where exams are allocated to timeslots. I will allow there to be H halls available at each timeslot. I will investigate varying the number of available timeslots T. Thus, the initial chromosome structure contains the features

#### Exam ID 
This is the ID of the examination. It will be a string, something like "00013" to represent the 13-th exam in the Exam-ID table. 

#### Hall ID
This is the ID of the hall. Simply, it will be a string in [01,02,...,10].

#### Timeslot ID
This is the ID of the timeslot. This is a string in range [0001,...,00T]. 


--------

Thus, we a chromosome which is [00015050009] represents the exam 00015, in hall 05 using the 9th time slot. This information is important for the fitness function.


### Fitness Function

I'm initially going to consider a very straight forward fitness function. To capture hard constraints, I will simply assign a -1 penalty when one is violated. I will add no soft constraints at this stage. Thus, the fitness at a given generation represents exactly how many violations to the hard constraints are present. I will deal with the following hard constraints:

#### Timetable Clash
For a given population of chromosomes, I will check how many students have clashing exams. This will be a simple count. I will iterate over the timesteps, and for each timestep over the students in the dataset. If a student has multiple exams for the given timestep, simply increment the total penalty for the generation by 1.

#### Exam not set 
It is also important that each exam is set, and each exam is set precisely once. Therefore, I will not mutate the Exam ID element of the chromosome. Thus, these remain fixed and the factors being mutated are the timetable slot (which must be filled) and the hall number (which again must be filled)

#### Each hall houses one exam
I will also add a penalty of -1 if one hall contains multiple exams. 





#### Process Outline

1. Create initial Population
2. Calculate Fitness of Population
3. Crossover Stage
4. Mutation Stage
5. Select Survivors
6. Go bcak to (2) until we meet termination criteria

In [302]:
import numpy as np
import pandas as pd

from random import *
import math

In [303]:
def initialize(data,T,H,G):
    print(f"Initializing Genetic Algorithm with: T = {T}, H = {H}, G = {G}")
    exams = list(data.columns)
    timeslots = ["T" + "0"*(2-len(str(x+1))) + str(x+1) for x in list(range(T))]
    halls = ["H" + "0"*(2-len(str(x+1))) + str(x+1) for x in list(range(H))]

    population = []

    for i in range(G):
        genes = []
        for index, exam in enumerate(exams): ##create G chromosomes, where each is a solution
            hall_idx = randint(0,H-1) 
            timeslot_idx = randint(0,T-1)

            genes.append([exam,halls[hall_idx],timeslots[timeslot_idx]])
        chromosome = pd.DataFrame(genes)
        chromosome.columns = ['exam','hall','timeslot']

        population.append(chromosome)
    return population,exams,timeslots,halls 

In [304]:
############################ FITNESS FUNCTION ###########################################

def get_fitness(population,data):
        print(f"Starting Fitness Check")
        ### Checking if there are any student clashes  
        pop_fitness = [] 

        for counter,chromosome in enumerate(population):
                chromosome_fitness = [] 
                
                for idx,row in data[:-1].iterrows():
                        student_slots = [] 
                        exams = list(row[row == 1].index)
                        for exam in exams:
                                exam_time = chromosome[chromosome.exam == exam]['timeslot'].values[0]
                                student_slots.append(exam_time)

                        dupes = [number for number in student_slots if student_slots.count(number) > 1]
                        chromosome_fitness.append(len(dupes) - len(np.unique(dupes)))

                pop_fitness.append(sum(chromosome_fitness))

                print(f"Computing Fitness for Chromosome {counter} of {len(population)}", end = "\r")

        return pop_fitness

def do_crossover(population,pop_fitness):
        print("Starting Crossover Process")
        scored_chromosomes = pd.DataFrame([population,pop_fitness]).T
        chosen_parents = list(scored_chromosomes.sort_values(1).head(int(len(scored_chromosomes)/2)).reset_index(drop = True)[0].values)

        parents = []
        flag = False

        while (flag == False):
                chosen_parent = chosen_parents[0]
                chosen_parents.pop(0)

                random_sample = randint(0,len(chosen_parents)-1)
                chosen_parent2 = chosen_parents[random_sample]
                chosen_parents.pop(random_sample)
                parents.append([chosen_parent,chosen_parent2])

                if len(chosen_parents) == 0:
                        flag = True
        return parents 

def perform_swaps(parents):
        mom,dad = parents[0]

        indices = mom.sample(n=int(len(mom)/2)).exam.values

        for index,item in enumerate(indices):
                dad_info = dad[dad.exam == item]
                dad_hall = dad_info.hall.values[0]
                dad_timeslot = dad_info.timeslot.values[0]

                mom_info = mom[mom.exam == item]
                mom_hall = mom_info.hall.values[0]
                mom_timeslot = mom_info.timeslot.values[0]

                dad.loc[dad_info.index,'hall'] = mom_hall
                dad.loc[dad_info.index,'timeslot'] = mom_timeslot

                mom.loc[mom_info.index,'hall'] = dad_hall
                mom.loc[mom_info.index,'timeslot'] = dad_timeslot

        return [mom,dad]


def mutate(parents,halls,timeslots,mutation_rate = 0.05,):
        print("Mutating")
        new_pairs = [] 
        print("Swapping Chromosomes")
        for parent_pair in parents:
                new_pairs.append(perform_swaps(parents))

        new_pairs = [item for sublist in new_pairs for item in sublist]
        full_population = new_pairs + new_pairs

        #performs mutation 
        new_population = []

        for sample_pop in full_population:
                change_rows = sample_pop.sample(int(len(sample_pop)*mutation_rate)).index

                for idx,row in sample_pop.iloc[change_rows].iterrows():
                        sample_pop.loc[idx,'hall'] = sample(halls,1)[0]
                        sample_pop.loc[idx,'timeslot'] = sample(timeslots,1)[0]
                new_population.append(sample_pop)
        return new_population

In [305]:
path = "../Data/Toronto/Processed"
file = "/conflicts_version2_ear-f-83.csv"

data = pd.read_csv(path+file)
data.replace(np.NaN,0,inplace= True)
data.columns = ["E" + "0"*(4-len(str(x))) + str(x) for x in data.columns]


G = 24 #size of each generation
Ngen = 10 #Number of Generations
T = 20 #number of time periods
H = 10 #number of available halls for each time period
mutation_rate = 0.05 #mutation rate
selection_ratio = 0.5 #Cross Over Proportion

def run_ga():
    population,exams,timeslots,halls = initialize(data,T,H,G)
    fitness = get_fitness(population,data)
    for generation in range(Ngen):
        print(f"Mean fitness: {np.mean(fitness)}")
        print(f"Minimum fitness: {np.min(fitness)}")
        parents = do_crossover(population,fitness)
        new_population = mutate(parents,halls,timeslots,mutation_rate = 0.05)
        fitness = get_fitness(new_population,data)



In [306]:
run_ga()

Initializing Genetic Algorithm with: T = 20, H = 10, G = 24
Starting Fitness Check
Mean fitness: 1174.7916666666667 23 of 24
Minimum fitness: 947
Starting Crossover Process
Mutating
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Starting Fitness Check
Mean fitness: 1199.0r Chromosome 23 of 24
Minimum fitness: 1163
Starting Crossover Process
Mutating
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Starting Fitness Check
Mean fitness: 1069.5r Chromosome 23 of 24
Minimum fitness: 1065
Starting Crossover Process
Mutating
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Starting Fitness Check
Mean fitness: 1080.0r Chromosome 23 of 24
Minimum fitness: 1032
Starting Crossover Process
Mutating
Swapping Chromosomes
Swapping Chromosomes
Swapping Chromosomes
Swapping